In [34]:
torch.cuda.empty_cache()

In [40]:
input = torch.rand(4, 400, 256).to(device)  # batch, num_frames, feature_dim
lengths = torch.randint(1, 200, (4,)).to(device)  # batch
output, lengths = emformer(input, lengths)

In [41]:
print(output,lengths)

tensor([[[ 0.0751,  1.1741,  0.2803,  ..., -1.3751,  1.7454,  2.6569],
         [ 0.4204,  1.7470,  0.2313,  ...,  0.1595,  1.5232, -0.7971],
         [ 1.3585, -0.3609,  0.6504,  ...,  1.2292,  1.7976, -1.1870],
         ...,
         [-0.6052, -0.3736,  1.1656,  ..., -0.2788, -1.1069, -0.5875],
         [-0.8173, -0.7149,  2.1800,  ..., -1.6249,  0.0324,  0.7269],
         [-0.2114, -0.6623,  0.5437,  ..., -0.4885,  1.0311, -1.4219]],

        [[-0.7416,  0.5183, -1.5463,  ...,  0.4775, -0.0608, -0.4215],
         [-0.8543, -1.1387,  0.3019,  ..., -0.0884,  1.5694,  0.9932],
         [-0.6936,  1.1782,  0.2838,  ..., -0.9359,  0.6605, -0.2664],
         ...,
         [ 0.2511, -0.6351,  0.8071,  ...,  0.9944,  1.4654,  0.7430],
         [-0.0368, -0.9818,  0.4863,  ...,  0.9570,  0.6864, -0.0708],
         [ 0.1201, -0.2853,  0.3010,  ..., -0.1932,  1.6423,  0.1028]],

        [[-0.9769, -0.6440,  2.0863,  ..., -0.6719, -0.7626,  0.9121],
         [-1.7482,  0.5123, -0.0846,  ..., -0

In [42]:
del output,lengths

In [44]:
input = torch.rand(1, 9, 256)
lengths = torch.ones(1) * 9
start = time.time()
output, lengths, states = emformer.infer(input.to(device), lengths.to(device),None)
end = time.time()
print('Eslasped',end-start)

Eslasped 0.01201772689819336


In [46]:
output.shape

torch.Size([1, 8, 256])

In [67]:
list(emformer.children())[0].kernel_size[0]

8

In [1]:
import torchaudio
from torchaudio.pipelines import EMFORMER_RNNT_BASE_LIBRISPEECH
import torch
# Non-streaming inference.
# Build feature extractor, decoder with RNN-T model, and token processor.
feature_extractor = EMFORMER_RNNT_BASE_LIBRISPEECH.get_feature_extractor()
decoder = EMFORMER_RNNT_BASE_LIBRISPEECH.get_decoder()
Downloading: "https://download.pytorch.org/torchaudio/models/emformer_rnnt_base_librispeech.pt"
token_processor = EMFORMER_RNNT_BASE_LIBRISPEECH.get_token_processor()
# Instantiate LibriSpeech dataset; retrieve waveform for first sample.
dataset = torchaudio.datasets.LIBRISPEECH("./data/librespeech", url="test-clean",download=True)


In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
decoder.to(device)

RNNTBeamSearch(
  (model): RNNT(
    (transcriber): _EmformerEncoder(
      (input_linear): Linear(in_features=80, out_features=128, bias=False)
      (time_reduction): _TimeReduction()
      (transformer): Emformer(
        (memory_op): AvgPool1d(kernel_size=(4,), stride=(4,), padding=(0,))
        (emformer_layers): ModuleList(
          (0-19): 20 x _EmformerLayer(
            (attention): _EmformerAttention(
              (emb_to_key_value): Linear(in_features=512, out_features=1024, bias=True)
              (emb_to_query): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (dropout): Dropout(p=0.1, inplace=False)
            (memory_op): AvgPool1d(kernel_size=(4,), stride=(4,), padding=(0,))
            (pos_ff): Sequential(
              (0): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (1): Linear(in_features=512, out_features=2048, bias=True)
       

In [4]:
waveform = next(iter(dataset))[0].squeeze()
with torch.no_grad():
    # Produce mel-scale spectrogram features.
    features, length = feature_extractor(waveform)
    # Generate top-10 hypotheses.
    hypotheses = decoder(features, length, 10)
# For top hypothesis, convert predicted tokens to text.
text = token_processor(hypotheses[0][0])
print(text,features)

he hoped there would be stew for dinner turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick peppered flour fat and sauce tensor([[-0.0557, -0.0557, -1.1367,  ..., -1.1762, -1.2420, -1.0377],
        [-0.1895, -0.1893, -0.2915,  ..., -0.7555, -0.7314, -0.7503],
        [-0.2962, -0.2958, -0.3840,  ..., -0.8993, -0.6641, -0.6368],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [5]:
waveform.shape

torch.Size([166960])

In [6]:
print(features.shape,length)

torch.Size([1048, 80]) tensor([1048])


In [7]:
token_processor

In [29]:
import torch
import torch.nn as nn


class RawHead(nn.Module):
    def __init__(self, hidden_size, num_layers, class_num=1, dropout=0.01):
        super().__init__()
        self.ff_dropout = dropout
        self.hidden_size = hidden_size
        self.feat = nn.Sequential(
            *[
                self._make_layer(hidden_size, hidden_size)
                for _ in range(num_layers - 1)
            ],
            nn.Linear(hidden_size, hidden_size)
        )
        self.head = nn.Sequential(
            nn.LayerNorm(self.hidden_size),
            nn.ReLU(),
            nn.Dropout(self.ff_dropout),
            nn.Linear(hidden_size, class_num),
        )

    def _make_layer(self, input_size, output_size):
        return nn.Sequential(
            nn.Linear(input_size, output_size),
            nn.LayerNorm(self.hidden_size),
            nn.ReLU(),
            nn.Dropout(self.ff_dropout),
        )

    def forward(self, inputs):
        out = self.feat(inputs)
        conf = self.head(out)
        return out,conf

In [42]:
head = RawHead(80,4,1)
head.eval()

RawHead(
  (feat): Sequential(
    (0): Sequential(
      (0): Linear(in_features=80, out_features=80, bias=True)
      (1): LayerNorm((80,), eps=1e-05, elementwise_affine=True)
      (2): ReLU()
      (3): Dropout(p=0.01, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=80, bias=True)
      (1): LayerNorm((80,), eps=1e-05, elementwise_affine=True)
      (2): ReLU()
      (3): Dropout(p=0.01, inplace=False)
    )
    (2): Sequential(
      (0): Linear(in_features=80, out_features=80, bias=True)
      (1): LayerNorm((80,), eps=1e-05, elementwise_affine=True)
      (2): ReLU()
      (3): Dropout(p=0.01, inplace=False)
    )
    (3): Linear(in_features=80, out_features=80, bias=True)
  )
  (head): Sequential(
    (0): LayerNorm((80,), eps=1e-05, elementwise_affine=True)
    (1): ReLU()
    (2): Dropout(p=0.01, inplace=False)
    (3): Linear(in_features=80, out_features=1, bias=True)
  )
)

In [45]:
import time
inp = torch.rand(2048,80)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features.to(device)
print(features.device)
start = time.time()
feat,conf = head(features)
hypotheses = decoder(feat.to(device), length, 10)
end = time.time()
print(feat.shape)
print(token_processor(hypotheses[0][0]))
print(end-start)
print(conf.sigmoid().median())

cpu

3.3725433349609375
tensor(0.4382, grad_fn=<MedianBackward0>)


In [ ]:
out

In [130]:
#!/usr/bin/env python3
"""Trains a SentencePiece model on transcripts across LibriSpeech train-clean-100, train-clean-360, and train-other-500.

Example:
python train_spm.py --librispeech-path ./datasets
"""

import io
import pathlib
from argparse import ArgumentParser, RawTextHelpFormatter

import sentencepiece as spm


def get_transcript_text(transcript_path):
    with open(transcript_path) as f:
        return [line.strip().split(" ", 1)[1].lower() for line in f]


def get_transcripts(dataset_path):
    transcript_paths = dataset_path.glob("*/*/*.trans.txt")
    merged_transcripts = []
    for path in transcript_paths:
        merged_transcripts += get_transcript_text(path)
    return merged_transcripts


def train_spm(input):
    model_writer = io.BytesIO()
    spm.SentencePieceTrainer.train(
        sentence_iterator=iter(input),
        model_writer=model_writer,
        vocab_size=90,
        model_type="bpe",
        input_sentence_size=-1,
        character_coverage=1.0,
        bos_id=0,
        pad_id=1,
        eos_id=2,
        unk_id=3,
    )
    return model_writer.getvalue()


def parse_args():
    parser = ArgumentParser(description=__doc__, formatter_class=RawTextHelpFormatter)
    parser.add_argument(
        "--librispeech-path",
        required=True,
        type=pathlib.Path,
        help="Path to LibriSpeech dataset.",
    )
    parser.add_argument(
        "--output-file",
        default=pathlib.Path("./spm_bpe_4096.model"),
        type=pathlib.Path,
        help="File to save model to. (Default: './spm_bpe_4096.model')",
    )
    return parser.parse_args()


def run_cli():
    args = parse_args()

    root = args.librispeech_path / "LibriSpeech"
    splits = ["train-clean-100", "train-clean-360", "train-other-500"]
    merged_transcripts = []
    for split in splits:
        path = pathlib.Path(root) / split
        merged_transcripts += get_transcripts(path)

    model = train_spm(merged_transcripts)

    with open(args.output_file, "wb") as f:
        f.write(model)

In [132]:
mdoel = train_spm(['สวัสดีครับ','เฮ้ย','ว่าไงหรอครับ','สบายดีครับ'])

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input_format: 
  model_prefix: 
  model_type: BPE
  vocab_size: 90
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 18446744073709551615
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 0
  eos_id: 2
  pad_id: 1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_noise_level: 0
  differential_privac